In [4]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd 
from selenium.webdriver.common.by import By 
from selenium.webdriver.chrome.service import Service
import os

style-scope ytd-rich-item-renderer
title //*[@id="video-title-link"]
views //*[@id="metadata-line"]/span[1]
time //*[@id="metadata-line"]/span[2]

In [5]:
url = "https://www.youtube.com/@krishnaik06/videos"
PATH = r'chromedriver.exe'

service = Service(executable_path=PATH)
wd = webdriver.Chrome(service=service)

In [6]:
wd.get(url)

In [7]:
videos = wd.find_elements(By.CLASS_NAME,"style-scope ytd-rich-item-renderer")

In [18]:
titles = []
for video in videos:
    title = video.find_element(By.XPATH,".//*[@id='video-title-link']").text

    titles.append(title)

titles

['Evidently AI Tutorial-Open Source ML Models Monitoring and Observability',
 'Is Transition From Mechanical Engineering To Data Science Possible? Shivam Transition Story Podcast',
 'BentoML Tutorial: Build Production Grade AI Applications',
 'Rohan Successful Transition-Freshers To MLOPS Lead With Equity @PWSkillsTech #datascience #ai',
 'Projjal Transition Story To Data Science With 380% Hike',
 'Tanisha Transition Story To Data Analytics Industry With 200% Hike',
 'Tavishi Transition Story To Data Science With 6 Offers And 170% Hike @PWSkillsTech',
 'End To End MLOPS Data Science Project Implementation With Deployment',
 'Data Versioning Hands On Tutorials With DVC For Data Science',
 'The Power Of MLFLOW And Dagshub-Open Source Data Science Project Collaboration',
 'Will AI Eat Your Job After ChatGPT And Code Interpreter- What You Should Do?',
 'The Power Of ChatGPT Plugins|Check This Powerful Demo',
 'The Future Of Coding-Code Interpreter With Chatgpt- Its Time To Make AI Your Fri